In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import re
import sys
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 250

In [2]:
sys.path.append('../')
import read_player_stats
import training_data

In [3]:
qb_game_df = pd.read_csv('../game_stats/qb_game_stats.csv')

In [4]:
qb_game_df = qb_game_df.drop(['Unnamed: 0'], axis = 1)
qb_game_df

,Name,Team,Games,PassComp,PassAtt,PassYards,PassTD,INT,RunAtt,RunYards,RunTD,FFP,FFPPG,Season,Week
0,AJFeeley,MIA,1,21,31,168,1,1,0,0,0,12.4,12.4,2004,1
1,AJFeeley,MIA,1,21,39,218,1,2,1,3,0,15.2,15.2,2004,2
2,AJFeeley,MIA,1,13,27,137,0,2,4,1,0,7.0,7.0,2004,3
3,AJFeeley,MIA,1,1,2,6,0,0,0,0,0,0.3,0.3,2004,5
4,AJFeeley,MIA,1,6,15,129,0,1,0,0,0,6.5,6.5,2004,9
5,AJFeeley,MIA,1,23,45,229,1,2,1,7,1,22.2,22.2,2004,11
6,AJFeeley,MIA,1,17,33,159,2,1,4,2,0,16.2,16.2,2004,12
7,AJFeeley,MIA,1,25,51,303,3,5,0,0,0,27.2,27.2,2004,13
8,AJFeeley,MIA,1,17,35,170,1,1,2,2,0,12.7,12.7,2004,14
9,AJFeeley,MIA,1,22,35,198,1,0,2,-2,0,13.7,13.7,2004,15


In [5]:
def game_dist(entry1, entry2):
    return (entry1['Season'] - entry2['Season'], entry1['Week'] - entry2['Week'])

def game_dist_number(entry1, entry2):
    return 17*(entry1['Season'] - entry2['Season']) + (entry1['Week'] - entry2['Week'])

In [6]:
#drop players with fewer than 20 games
qb_game_df = qb_game_df.groupby('Name').filter(lambda x: len(x) > 20)

In [7]:
orig = {'Season': 2004, 'Week': 1}

In [24]:
qb_game_df['WeekID'] = 17*(qb_game_df['Season'] - orig['Season']) + (qb_game_df['Week'] - orig['Week'])

/usr/local/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
qb_game_df

,Name,Team,Games,PassComp,PassAtt,PassYards,PassTD,INT,RunAtt,RunYards,RunTD,FFP,FFPPG,Season,Week,WeekID
0,AJFeeley,MIA,1,21,31,168,1,1,0,0,0,12.4,12.4,2004,1,0
1,AJFeeley,MIA,1,21,39,218,1,2,1,3,0,15.2,15.2,2004,2,1
2,AJFeeley,MIA,1,13,27,137,0,2,4,1,0,7.0,7.0,2004,3,2
3,AJFeeley,MIA,1,1,2,6,0,0,0,0,0,0.3,0.3,2004,5,4
4,AJFeeley,MIA,1,6,15,129,0,1,0,0,0,6.5,6.5,2004,9,8
5,AJFeeley,MIA,1,23,45,229,1,2,1,7,1,22.2,22.2,2004,11,10
6,AJFeeley,MIA,1,17,33,159,2,1,4,2,0,16.2,16.2,2004,12,11
7,AJFeeley,MIA,1,25,51,303,3,5,0,0,0,27.2,27.2,2004,13,12
8,AJFeeley,MIA,1,17,35,170,1,1,2,2,0,12.7,12.7,2004,14,13
9,AJFeeley,MIA,1,22,35,198,1,0,2,-2,0,13.7,13.7,2004,15,14


In [10]:
temp_df = qb_game_df[qb_game_df['Name'] == 'AaronRodgers']
temp_df

,Name,Team,Games,PassComp,PassAtt,PassYards,PassTD,INT,RunAtt,RunYards,RunTD,FFP,FFPPG,Season,Week,WeekID
61,AaronRodgers,GB,1,1,1,0,0,0,0,0,0,0.0,0.0,2005,5,21
62,AaronRodgers,GB,1,8,15,65,0,1,1,8,0,4.1,4.1,2005,15,31
63,AaronRodgers,GB,1,0,0,0,0,0,1,-1,0,-0.1,-0.1,2005,17,33
64,AaronRodgers,GB,1,2,3,14,0,0,0,0,0,0.7,0.7,2006,4,37
65,AaronRodgers,GB,1,4,12,32,0,0,2,11,0,2.7,2.7,2006,11,44
66,AaronRodgers,GB,1,2,2,17,0,0,2,-1,0,0.8,0.8,2007,10,60
67,AaronRodgers,GB,1,18,26,201,1,0,5,30,0,17.1,17.1,2007,13,63
68,AaronRodgers,GB,1,18,22,178,1,0,8,35,1,22.4,22.4,2008,1,68
69,AaronRodgers,GB,1,24,38,328,3,0,4,25,0,30.9,30.9,2008,2,69
70,AaronRodgers,GB,1,22,39,290,0,0,5,10,1,21.5,21.5,2008,3,70


In [11]:
temp_df = temp_df.sort(columns='WeekID', ascending=False)
temp_df.reset_index(drop=True, inplace=True)
temp_df

,Name,Team,Games,PassComp,PassAtt,PassYards,PassTD,INT,RunAtt,RunYards,RunTD,FFP,FFPPG,Season,Week,WeekID
0,AaronRodgers,GB,1,17,22,226,2,0,2,14,1,26.7,26.7,2014,17,186
1,AaronRodgers,GB,1,31,40,318,1,0,0,0,0,19.9,19.9,2014,16,185
2,AaronRodgers,GB,1,17,42,185,0,2,3,27,0,12.0,12.0,2014,15,184
3,AaronRodgers,GB,1,24,36,327,3,0,5,28,0,31.2,31.2,2014,14,183
4,AaronRodgers,GB,1,24,38,368,2,0,5,22,0,28.6,28.6,2014,13,182
5,AaronRodgers,GB,1,19,29,209,2,0,6,34,0,21.9,21.9,2014,12,181
6,AaronRodgers,GB,1,22,36,341,3,0,3,32,0,32.3,32.3,2014,11,180
7,AaronRodgers,GB,1,18,27,315,6,0,0,0,0,39.8,39.8,2014,10,179
8,AaronRodgers,GB,1,28,39,418,1,2,2,21,1,33.0,33.0,2014,8,177
9,AaronRodgers,GB,1,19,22,255,3,0,3,21,0,26.9,26.9,2014,7,176


In [12]:
arr = np.array(temp_df.drop(['Name', 'Team', 'Games', 'Season', 'Week', 'WeekID', 'FFPPG'], axis=1))

In [13]:
tot_games,nfeat = arr.shape    
windowlen = 20*nfeat

In [14]:
flat = arr.flatten()
flat.shape

data_feat = flat[:windowlen]
data_feat.shape

for i in range(tot_games - windowlen/nfeat + 1):
    index = i*nfeat
    data_feat = np.vstack([data_feat, flat[index:index+windowlen]])

In [15]:
data_feat.shape

(92, 180)

In [16]:
len(data_feat)/(nfeat*20)

0

In [17]:
def make_game_data(df):
    df = df.sort(columns='WeekID', ascending=False)
    df.reset_index(drop=True, inplace=True)
    arr = np.array(df.drop(['Name', 'Team', 'Games', 'Season', 'Week', 'FFPPG'], axis=1))
    tot_games,nfeat = arr.shape    
    windowlen = 20*nfeat
    flat = arr.flatten()
    
    label = np.array([-1])
    data_feat = np.concatenate([label, flat[:windowlen]], axis=1) 
        
    for i in range(tot_games - windowlen/nfeat + 1):
        if i==0: 
            continue
        index = i*nfeat
        if (index - 1) >= 0 :
            label = flat[index - 1]
        else:
            label = np.array([-1])
        row = np.hstack([label, flat[index:index+windowlen]])
        data_feat = np.vstack([data_feat, row])
        
    return data_feat

In [18]:
stats_cols = temp_df.drop(['Name', 'Team', 'Games', 'Season', 'Week','FFPPG'], axis=1).columns
game_stats_cols = []
for i in range(20):
    game_stats_cols = game_stats_cols + [(col + '_' + str(i)) for col in stats_cols]
game_stats_cols

['PassComp_0',
 'PassAtt_0',
 'PassYards_0',
 'PassTD_0',
 'INT_0',
 'RunAtt_0',
 'RunYards_0',
 'RunTD_0',
 'FFP_0',
 'WeekID_0',
 'PassComp_1',
 'PassAtt_1',
 'PassYards_1',
 'PassTD_1',
 'INT_1',
 'RunAtt_1',
 'RunYards_1',
 'RunTD_1',
 'FFP_1',
 'WeekID_1',
 'PassComp_2',
 'PassAtt_2',
 'PassYards_2',
 'PassTD_2',
 'INT_2',
 'RunAtt_2',
 'RunYards_2',
 'RunTD_2',
 'FFP_2',
 'WeekID_2',
 'PassComp_3',
 'PassAtt_3',
 'PassYards_3',
 'PassTD_3',
 'INT_3',
 'RunAtt_3',
 'RunYards_3',
 'RunTD_3',
 'FFP_3',
 'WeekID_3',
 'PassComp_4',
 'PassAtt_4',
 'PassYards_4',
 'PassTD_4',
 'INT_4',
 'RunAtt_4',
 'RunYards_4',
 'RunTD_4',
 'FFP_4',
 'WeekID_4',
 'PassComp_5',
 'PassAtt_5',
 'PassYards_5',
 'PassTD_5',
 'INT_5',
 'RunAtt_5',
 'RunYards_5',
 'RunTD_5',
 'FFP_5',
 'WeekID_5',
 'PassComp_6',
 'PassAtt_6',
 'PassYards_6',
 'PassTD_6',
 'INT_6',
 'RunAtt_6',
 'RunYards_6',
 'RunTD_6',
 'FFP_6',
 'WeekID_6',
 'PassComp_7',
 'PassAtt_7',
 'PassYards_7',
 'PassTD_7',
 'INT_7',
 'RunAtt_7',
 '

In [19]:
temp_df = qb_game_df[qb_game_df['Name'] == 'AaronRodgers']
AR_data = make_game_data(temp_df)
AR_df = pd.DataFrame(AR_data, columns=(['Label'] + game_stats_cols))
AR_df['Name'] = 'AaronRodgers'
AR_df.head()

,Label,PassComp_0,PassAtt_0,PassYards_0,PassTD_0,INT_0,RunAtt_0,RunYards_0,RunTD_0,FFP_0,WeekID_0,PassComp_1,PassAtt_1,PassYards_1,PassTD_1,INT_1,RunAtt_1,RunYards_1,RunTD_1,FFP_1,WeekID_1,PassComp_2,PassAtt_2,PassYards_2,PassTD_2,INT_2,RunAtt_2,RunYards_2,RunTD_2,FFP_2,WeekID_2,PassComp_3,PassAtt_3,PassYards_3,PassTD_3,INT_3,RunAtt_3,RunYards_3,RunTD_3,FFP_3,WeekID_3,PassComp_4,PassAtt_4,PassYards_4,PassTD_4,INT_4,RunAtt_4,RunYards_4,RunTD_4,FFP_4,WeekID_4,PassComp_5,PassAtt_5,PassYards_5,PassTD_5,INT_5,RunAtt_5,RunYards_5,RunTD_5,FFP_5,WeekID_5,PassComp_6,PassAtt_6,PassYards_6,PassTD_6,INT_6,RunAtt_6,RunYards_6,RunTD_6,FFP_6,WeekID_6,PassComp_7,PassAtt_7,PassYards_7,PassTD_7,INT_7,RunAtt_7,RunYards_7,RunTD_7,FFP_7,WeekID_7,PassComp_8,PassAtt_8,PassYards_8,PassTD_8,INT_8,RunAtt_8,RunYards_8,RunTD_8,FFP_8,WeekID_8,PassComp_9,PassAtt_9,PassYards_9,PassTD_9,INT_9,RunAtt_9,RunYards_9,RunTD_9,FFP_9,WeekID_9,PassComp_10,PassAtt_10,PassYards_10,PassTD_10,INT_10,RunAtt_10,RunYards_10,RunTD_10,FFP_10,WeekID_10,PassComp_11,PassAtt_11,PassYards_11,PassTD_11,INT_11,RunAtt_11,RunYards_11,RunTD_11,FFP_11,WeekID_11,PassComp_12,PassAtt_12,PassYards_12,PassTD_12,INT_12,RunAtt_12,RunYards_12,RunTD_12,FFP_12,WeekID_12,PassComp_13,PassAtt_13,PassYards_13,PassTD_13,INT_13,RunAtt_13,RunYards_13,RunTD_13,FFP_13,WeekID_13,PassComp_14,PassAtt_14,PassYards_14,PassTD_14,INT_14,RunAtt_14,RunYards_14,RunTD_14,FFP_14,WeekID_14,PassComp_15,PassAtt_15,PassYards_15,PassTD_15,INT_15,RunAtt_15,RunYards_15,RunTD_15,FFP_15,WeekID_15,PassComp_16,PassAtt_16,PassYards_16,PassTD_16,INT_16,RunAtt_16,RunYards_16,RunTD_16,FFP_16,WeekID_16,PassComp_17,PassAtt_17,PassYards_17,PassTD_17,INT_17,RunAtt_17,RunYards_17,RunTD_17,FFP_17,WeekID_17,PassComp_18,PassAtt_18,PassYards_18,PassTD_18,INT_18,RunAtt_18,RunYards_18,RunTD_18,FFP_18,WeekID_18,PassComp_19,PassAtt_19,PassYards_19,PassTD_19,INT_19,RunAtt_19,RunYards_19,RunTD_19,FFP_19,WeekID_19,Name
0,-1,17,22,226,2,0,2,14,1,26.7,186,31,40,318,1,0,0,0,0,19.9,185,17,42,185,0,2,3,27,0,12.0,184,24,36,327,3,0,5,28,0,31.2,183,24,38,368,2,0,5,22,0,28.6,182,19,29,209,2,0,6,34,0,21.9,181,22,36,341,3,0,3,32,0,32.3,180,18,27,315,6,0,0,0,0,39.8,179,28,39,418,1,2,2,21,1,33.0,177,19,22,255,3,0,3,21,0,26.9,176,24,42,264,3,0,7,34,0,28.6,175,12,17,156,3,0,0,0,0,19.8,174,22,28,302,4,0,1,8,0,31.9,173,16,27,162,1,0,0,0,0,12.1,172,25,42,346,3,0,6,28,0,32.1,171,23,33,189,1,1,0,0,0,13.5,170,25,39,318,2,2,1,5,0,24.4,169,1,2,27,0,0,1,9,0,2.3,161,24,29,285,2,0,6,31,0,25.4,160,25,36,260,3,0,4,12,0,26.2,159,AaronRodgers
1,186,31,40,318,1,0,0,0,0,19.9,185,17,42,185,0,2,3,27,0,12.0,184,24,36,327,3,0,5,28,0,31.2,183,24,38,368,2,0,5,22,0,28.6,182,19,29,209,2,0,6,34,0,21.9,181,22,36,341,3,0,3,32,0,32.3,180,18,27,315,6,0,0,0,0,39.8,179,28,39,418,1,2,2,21,1,33.0,177,19,22,255,3,0,3,21,0,26.9,176,24,42,264,3,0,7,34,0,28.6,175,12,17,156,3,0,0,0,0,19.8,174,22,28,302,4,0,1,8,0,31.9,173,16,27,162,1,0,0,0,0,12.1,172,25,42,346,3,0,6,28,0,32.1,171,23,33,189,1,1,0,0,0,13.5,170,25,39,318,2,2,1,5,0,24.4,169,1,2,27,0,0,1,9,0,2.3,161,24,29,285,2,0,6,31,0,25.4,160,25,36,260,3,0,4,12,0,26.2,159,17,32,315,1,1,5,21,0,21.9,158,AaronRodgers
2,185,17,42,185,0,2,3,27,0,12.0,184,24,36,327,3,0,5,28,0,31.2,183,24,38,368,2,0,5,22,0,28.6,182,19,29,209,2,0,6,34,0,21.9,181,22,36,341,3,0,3,32,0,32.3,180,18,27,315,6,0,0,0,0,39.8,179,28,39,418,1,2,2,21,1,33.0,177,19,22,255,3,0,3,21,0,26.9,176,24,42,264,3,0,7,34,0,28.6,175,12,17,156,3,0,0,0,0,19.8,174,22,28,302,4,0,1,8,0,31.9,173,16,27,162,1,0,0,0,0,12.1,172,25,42,346,3,0,6,28,0,32.1,171,23,33,189,1,1,0,0,0,13.5,170,25,39,318,2,2,1,5,0,24.4,169,1,2,27,0,0,1,9,0,2.3,161,24,29,285,2,0,6,31,0,25.4,160,25,36,260,3,0,4,12,0,26.2,159,17,32,315,1,1,5,21,0,21.9,158,20,30,274,1,0,5,8,0,18.5,157,AaronRodgers
3,184,24,36,327,3,0,5,28,0,31.2,183,24,38,368,2,0,5,22,0,28.6,182,19,29,209,2,0,6,34,0,21.9,181,22,36,341,3,0,3,32,0,32.3,180,18,27,315,6,0,0,0,0,39.8,179,28,39,418,1,2,2,21,1,33.0,177,19,22,255,3,0,3,21,0,26.9,176,24,42,264,3,0,7,34,0,28.6,175,12,

In [20]:
names = qb_game_df['Name'].unique()
data_list = []
for name in names:
    feat_arr = make_game_data(qb_game_df[qb_game_df.Name == name])
    feat_df = pd.DataFrame(feat_arr, columns=(['Label'] + game_stats_cols))
    feat_df['Name'] = name
    data_list.append(feat_df)

stacked_data = pd.concat(data_list)

In [21]:
stacked_data.head(10)

,Label,PassComp_0,PassAtt_0,PassYards_0,PassTD_0,INT_0,RunAtt_0,RunYards_0,RunTD_0,FFP_0,WeekID_0,PassComp_1,PassAtt_1,PassYards_1,PassTD_1,INT_1,RunAtt_1,RunYards_1,RunTD_1,FFP_1,WeekID_1,PassComp_2,PassAtt_2,PassYards_2,PassTD_2,INT_2,RunAtt_2,RunYards_2,RunTD_2,FFP_2,WeekID_2,PassComp_3,PassAtt_3,PassYards_3,PassTD_3,INT_3,RunAtt_3,RunYards_3,RunTD_3,FFP_3,WeekID_3,PassComp_4,PassAtt_4,PassYards_4,PassTD_4,INT_4,RunAtt_4,RunYards_4,RunTD_4,FFP_4,WeekID_4,PassComp_5,PassAtt_5,PassYards_5,PassTD_5,INT_5,RunAtt_5,RunYards_5,RunTD_5,FFP_5,WeekID_5,PassComp_6,PassAtt_6,PassYards_6,PassTD_6,INT_6,RunAtt_6,RunYards_6,RunTD_6,FFP_6,WeekID_6,PassComp_7,PassAtt_7,PassYards_7,PassTD_7,INT_7,RunAtt_7,RunYards_7,RunTD_7,FFP_7,WeekID_7,PassComp_8,PassAtt_8,PassYards_8,PassTD_8,INT_8,RunAtt_8,RunYards_8,RunTD_8,FFP_8,WeekID_8,PassComp_9,PassAtt_9,PassYards_9,PassTD_9,INT_9,RunAtt_9,RunYards_9,RunTD_9,FFP_9,WeekID_9,PassComp_10,PassAtt_10,PassYards_10,PassTD_10,INT_10,RunAtt_10,RunYards_10,RunTD_10,FFP_10,WeekID_10,PassComp_11,PassAtt_11,PassYards_11,PassTD_11,INT_11,RunAtt_11,RunYards_11,RunTD_11,FFP_11,WeekID_11,PassComp_12,PassAtt_12,PassYards_12,PassTD_12,INT_12,RunAtt_12,RunYards_12,RunTD_12,FFP_12,WeekID_12,PassComp_13,PassAtt_13,PassYards_13,PassTD_13,INT_13,RunAtt_13,RunYards_13,RunTD_13,FFP_13,WeekID_13,PassComp_14,PassAtt_14,PassYards_14,PassTD_14,INT_14,RunAtt_14,RunYards_14,RunTD_14,FFP_14,WeekID_14,PassComp_15,PassAtt_15,PassYards_15,PassTD_15,INT_15,RunAtt_15,RunYards_15,RunTD_15,FFP_15,WeekID_15,PassComp_16,PassAtt_16,PassYards_16,PassTD_16,INT_16,RunAtt_16,RunYards_16,RunTD_16,FFP_16,WeekID_16,PassComp_17,PassAtt_17,PassYards_17,PassTD_17,INT_17,RunAtt_17,RunYards_17,RunTD_17,FFP_17,WeekID_17,PassComp_18,PassAtt_18,PassYards_18,PassTD_18,INT_18,RunAtt_18,RunYards_18,RunTD_18,FFP_18,WeekID_18,PassComp_19,PassAtt_19,PassYards_19,PassTD_19,INT_19,RunAtt_19,RunYards_19,RunTD_19,FFP_19,WeekID_19,Name
0,-1,12,22,156,0,1,0,0,0,7.8,131,0,0,0,0,0,0,0,0,0.0,128,20,37,175,1,0,2,5,0,13.3,126,20,33,196,0,1,1,-1,0,9.7,125,1,5,21,0,0,0,0,0,1.1,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,16.4,63,27,42,345,3,3,1,2,0,29.5,62,13,19,116,1,1,2,7,0,10.5,61,22,33,321,3,0,1,3,0,28.4,50,4,5,21,0,0,0,0,0,1.1,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,12.8,15,22,35,198,1,0,2,-2,0,13.7,14,17,35,170,1,1,2,2,0,12.7,13,25,51,303,3,5,0,0,0,27.2,12,17,33,159,2,1,4,2,0,16.2,11,23,45,229,1,2,1,7,1,22.2,10,6,15,129,0,1,0,0,0,6.5,8,AJFeeley
1,131,0,0,0,0,0,0,0,0,0.0,128,20,37,175,1,0,2,5,0,13.3,126,20,33,196,0,1,1,-1,0,9.7,125,1,5,21,0,0,0,0,0,1.1,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,16.4,63,27,42,345,3,3,1,2,0,29.5,62,13,19,116,1,1,2,7,0,10.5,61,22,33,321,3,0,1,3,0,28.4,50,4,5,21,0,0,0,0,0,1.1,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,12.8,15,22,35,198,1,0,2,-2,0,13.7,14,17,35,170,1,1,2,2,0,12.7,13,25,51,303,3,5,0,0,0,27.2,12,17,33,159,2,1,4,2,0,16.2,11,23,45,229,1,2,1,7,1,22.2,10,6,15,129,0,1,0,0,0,6.5,8,1,2,6,0,0,0,0,0,0.3,4,AJFeeley
2,128,20,37,175,1,0,2,5,0,13.3,126,20,33,196,0,1,1,-1,0,9.7,125,1,5,21,0,0,0,0,0,1.1,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,16.4,63,27,42,345,3,3,1,2,0,29.5,62,13,19,116,1,1,2,7,0,10.5,61,22,33,321,3,0,1,3,0,28.4,50,4,5,21,0,0,0,0,0,1.1,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,12.8,15,22,35,198,1,0,2,-2,0,13.7,14,17,35,170,1,1,2,2,0,12.7,13,25,51,303,3,5,0,0,0,27.2,12,17,33,159,2,1,4,2,0,16.2,11,23,45,229,1,2,1,7,1,22.2,10,6,15,129,0,1,0,0,0,6.5,8,1,2,6,0,0,0,0,0,0.3,4,13,27,137,0,2,4,1,0,7.0,2,AJFeeley
3,126,20,33,196,0,1,1,-1,0,9.7,125,1,5,21,0,0,0,0,0,1.1,119,0,0,0,0,0,0,0,0,0.0,64,19,42,220,1,4,4,14,0,16.4,63,27,42,345,3,3,1,2,0,29.5,62,13,19,116,1,1,2,7,0,10.5,61,22,33,321,3,0,1,3,0,28.4,50,4,5,21,0,0,0,0,0,1.1,45,0,0,0,0,0,0,0,0,0.0,31,0,0,0,0,0,0,0,0,0.0,17,25,43,176,1,0,0,0,0,12.8,15,22,35,198,1,0,2,-2,0,13.7,14,17,35,170,1,1,2,2,0,12.7,13,25,51,303,3,5,0,0,0,27.2,12,17,33,159,2,1,4,2,0,16.2,11,23,45,229,1,2,1,7,1,22